# MDA: Post-procesamiento

Este notebook describe los métodos de post-procesamiento que fueron aplicados sobre los resultados de las predicciones del modelo de U-Net.

## Poligonizado

La siguiente función aplica una rutina de poligonización sobre los resultados de la predicción del modelo y genera un archivo vectorial en formato GeoPackage (GPKG). La rutina aplica `gdal_polygonize.py` a cada chip resultado generando un GPKG para cada chip, y luego une todos estos archivos en uno solo, de manera eficiente.

Antes de unirlos también aplica un umbral sobre los valores de los rásteres, que en este caso representan la probabilidad (valores entre 0 y 1).

In [2]:
from meduy.postprocess.filter import filter_by_max_prob
from meduy.postprocess.polygonize import polygonize
import os

In [3]:
filter_by_max_prob(input_dir='data/data_results/vuelo/4D/600_600/'
                   output_dir='data/data_results/vuelo/4D/600_600_filtered/',
                   threshold=0.4)

100%|█████████████████████████████████| 228835/228835 [03:04<00:00, 1243.06it/s]


In [4]:
input_path = os.path.join('data/data_results','vuelo','4D', '600_600_filtered')
output_path = "data/results/asentamientos_vuelo_4D_600_600.gpkg"

polygonize(threshold=0.4,
           input_dir=input_path,
           output=output_path)

100%|██████████| 55670/55670 [3:13:14<00:00,  4.80it/s]   


OSError: [Errno 28] No space left on device: '/tmp/tmp6u4ew1us/groups'

In [ ]:
filter_by_max_prob(input_dir='data/data_results/vuelo/5D/600_600/'
                   output_dir='data/data_results/vuelo/5D/600_600_filtered/',
                   threshold=0.4)

## Filtrar por área mínima

Como último paso, filtramos aquellos polígonos de area inferior a 300 m², dado que consideramos que son falsos positivos. Para esto utilizamos `ogr2ogr` y una consulta SQL. El resultado se guarda en `data/results/predict_300.gpkg`

In [ ]:
min_area = 300
input_path = "data/results/predict.gpkg"
output_path = "data/results/predict_300.gpkg"

In [ ]:
!ogr2ogr \
    -s_srs EPSG:5382 \
    -t_srs EPSG:5382 \
    -f "GPKG" \
    -sql "SELECT * FROM results m WHERE ST_Area(geom) > $min_area" \
    -dialect SQLITE \
    -nln results \
    $input_path
    $output_path

Opcionalmente, se presenta una manera de poder extraer la cantidad de polígonos resultado de la predicción del modelo. Cabe destacar que previamente se han elegido parámetros de área mínima en metros cuadrados y umbral de probabilidad mínima, que afectan los resultados de este cálculo.

Contemplar el nombre de archivo para poder ejecutar correctamente este comando. El hecho de calcular las probabilidades como "dn/255.0" refiere a que las imágenes están en formato Byte, que expresa como máximo 255 y como mínimo valor 0.

In [ ]:
!ogrinfo \
-sql "SELECT dn/255.0 as probabilidad, COUNT(*) as cantidad,  FROM results GROUP BY probabilidad" \
-dialect sqlite \
results_3d_remesa1.gpkg